# https://www.youtube.com/watch?v=NkUA2Q5gQNk&list=PL2IsFZBGM_IHCl9zhRVC1EXTomkEp_1zm&index=10

In [14]:
from pyspark.sql import SparkSession
from  pyspark.sql import Window
import pyspark.sql.functions as F
import os

In [2]:
os.environ['SPARK_HOME'] = r'C:\spark\spark-3.5.4-bin-hadoop3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [3]:
spark = (
    SparkSession
    .builder
    .appName("PySpark Zero to Hero")
    .master("local[*]")
    .config("spark.executor.memory", "16g")
    .config("spark.driver.memory", "16g")
    .config("spark.executor.cores", "4")
    .config("spark.sql.shuffle.partitions", "80")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.minExecutors", "2")
    .config("spark.dynamicAllocation.initialExecutors", "24")
    .config("spark.dynamicAllocation.maxExecutors", "50")
    .config("spark.shuffle.service.enabled", "true")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)

In [4]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

In [5]:
emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [6]:
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [7]:
emp_unique = emp.distinct()

In [10]:
emp_unique.show(5)

+-----------+-------------+----------+---+------+------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|
+-----------+-------------+----------+---+------+------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|
+-----------+-------------+----------+---+------+------+----------+
only showing top 5 rows



In [11]:
emp_dept_id = emp.select(
    'department_id'
).distinct()

In [12]:
emp_dept_id.show()

+-------------+
|department_id|
+-------------+
|          101|
|          102|
|          103|
|          104|
|          105|
|          106|
|          107|
+-------------+



In [22]:
window_spec = Window.partitionBy(F.col('department_id')).orderBy(F.col('salary').desc())

max_func = F.max(F.col('salary')).over(window_spec)

emp_1 = emp.withColumn('max_salary', max_func)

In [24]:
emp_1.show(5)

+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|max_salary|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|     70000|
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|     70000|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|     70000|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|     55000|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|     55000|
+-----------+-------------+-------------+---+------+------+----------+----------+
only showing top 5 rows



In [31]:
window_spec = Window.partitionBy(F.col('department_id')).orderBy(F.col('salary').desc())

rn = F.row_number().over(window_spec)

emp_2 = emp.withColumn(
    'rn', rn
).where('rn = 2')

In [32]:
emp_2.show(5)

+-----------+-------------+----------+---+------+------+----------+---+
|employee_id|department_id|      name|age|gender|salary| hire_date| rn|
+-----------+-------------+----------+---+------+------+----------+---+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|  2|
|        020|          102| Grace Kim| 32|Female| 53000|2018-11-01|  2|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|  2|
|        018|          104| Nancy Liu| 29|Female| 50000|2017-06-01|  2|
|        012|          105|Susan Chen| 31|Female| 54000|2017-02-15|  2|
+-----------+-------------+----------+---+------+------+----------+---+
only showing top 5 rows



In [36]:
emp_3 = emp.withColumn(
    'rn', F.expr('row_number() over(partition by department_id order by salary desc)')
).where('rn = 2')

In [37]:
emp_3.show(5)

+-----------+-------------+----------+---+------+------+----------+---+
|employee_id|department_id|      name|age|gender|salary| hire_date| rn|
+-----------+-------------+----------+---+------+------+----------+---+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|  2|
|        020|          102| Grace Kim| 32|Female| 53000|2018-11-01|  2|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|  2|
|        018|          104| Nancy Liu| 29|Female| 50000|2017-06-01|  2|
|        012|          105|Susan Chen| 31|Female| 54000|2017-02-15|  2|
+-----------+-------------+----------+---+------+------+----------+---+
only showing top 5 rows



In [38]:
spark.stop()